In [1]:
import json

from tqdm import tqdm

from src.config import Config

from src.utils.llm import call_llm, get_model_source


info = [{'recipe_chef': 'Aurora Stellaris',
  'recipe_name': 'Nebulosa Celestiale alla Stellaris',
  'recipe_restaurant': 'Anima Cosmica',
  'recipe_text': 'Lasciatevi trasportare in un viaggio attraverso le '
                 'profondità cosmiche con la nostra "Nebulosa Celestiale alla '
                 'Stellaris", una sinfonia culinaria che fonde la magia delle '
                 "stelle con l'arte della cucina quantistica. Al centro del "
                 'piatto brilla un arrosto di Carne di Balena Spaziale, '
                 'sapientemente cotto utilizzando microonde entropiche '
                 'sincronizzate per ottenere una superficie croccante e dorata '
                 'che racchiude un cuore succulento.\n'
                 '\n'
                 'Accanto, la carne di Mucca, teneramente preparata con la '
                 'cottura a vapore termocinetica multipla, si fonde '
                 "armoniosamente con funghi dell'Etere fluttuanti, esaltando "
                 'un delicato profumo che solletica i sensi. Un contorno di '
                 'Riso di Cassandra cuoce con vapore a flusso di particelle '
                 'isoarmoniche, ogni singolo chicco brilla come una stella '
                 'danzante, conservando i suoi nutrienti e colori '
                 'translucidi.\n'
                 '\n'
                'Il sapore imprevedibile delle Shard di Materia Oscura è '
                 "catturato in un'infusione sublime, avvolgendo tutto in "
                 "un'atmosfera di meraviglia gravitazionale. Sorprendetevi con "
                 'delle finissime teste di Idra, immerse in un consommé '
                 'rigenerativo, la cui preparazione segue tecniche di taglio a '
                 'risonanza sonica rigenerativa per assicurarne la sicurezza e '
                 'il sapore impeccabile.\n'
                 '\n'
                 "A completare questa opera d'arte, il nostro Pane di Luce "
                 'irradia energia dorata, pronto a raccogliere ogni sapore con '
                 'la sua morbidezza. E, come culmine di questo viaggio '
                 'stellare, una degustazione di Biscotti della Galassia ruota '
                 'allegramente intorno al piatto; scaglie di stelle e zucchero '
                 'cosmico, accanto a un velato sentore di spezie Melange, che '
                 "prolungano l'esperienza con un dolce indugio presciente che "
                 "eleva l'anima oltre le stelle.\n"
                 '\n'
                 '## Ingredienti\n'
                 '\n'
                 '- Shard di Materia Oscura\n'
                 '- Carne di Balena spaziale\n'
                 '- Carne di Mucca\n'
                 '- Teste di Idra\n'
                 '- Riso di Cassandra\n'
                 '- Biscotti della Galassia\n'
                 '- Pane di Luce\n'
                 "- Funghi dell'Etere\n"
                 '- Spezie Melange\n'
               '\n'
                 '## Tecniche\n'
                 '\n'
                 '- Cottura a Vapore con Flusso di Particelle Isoarmoniche\n'
                 '- Cottura a Vapore Termocinetica Multipla\n'
                 '- Taglio a Risonanza Sonica Rigenerativa\n'
                 '- Cottura con Microonde Entropiche Sincronizzate'
                 }]




provider = Config.provider
model = Config.model
output_model_str = get_model_source("src.datamodels", "DishRecipe")

output = []
for request in tqdm(info):
    #print(request)
    recipe_text = request['recipe_text']
    system_message = Config.system_message_template_recipes.format(
        output_model_str=output_model_str
    )
    message = Config.message_template_recipes.format(dish_recipe=recipe_text)

    try:
        response = call_llm(
            message=message,
            sys_message=system_message,
            model=f"{provider}:{model}",
            json_output=True,
        )
        print(response)
        response = json.loads(response)
        output_tmp = {"input_text": recipe_text, "output_JSON": response}
        # print(response)
        # print(recipe_text)
        output.append(output_tmp)
    except Exception:
        output.append({"input_text": recipe_text, "output_JSON": "Error"})
    



100%|██████████| 1/1 [00:02<00:00,  2.26s/it]

{
   "recipe_ingredients": [
       "Shard di Materia Oscura",
       "Carne di Balena spaziale",
       "Carne di Mucca",
       "Teste di Idra",
       "Riso di Cassandra",
       "Biscotti della Galassia",
       "Pane di Luce",
       "Funghi dell'Etere",
       "Spezie Melange"
   ],
   "recipe_techniques": [
       "Cottura a Vapore con Flusso di Particelle Isoarmoniche",
       "Cottura a Vapore Termocinetica Multipla",
       "Taglio a Risonanza Sonica Rigenerativa",
       "Cottura con Microonde Entropiche Sincronizzate"
   ]
}


In [1]:
import json
from pathlib import Path
from typing import Dict, List
import json
from tqdm import tqdm
from src.config import Config
from src.utils.llm import call_llm, get_model_source

example_data = [
    {
        "recipe_name": "Nebulosa Celestiale alla Stellaris",
        "recipe_raw_text": """
        Lasciatevi trasportare in un viaggio attraverso le profondità cosmiche con la nostra “Nebulosa Celestiale alla Stellaris”, una sinfonia culinaria che
        fonde la magia delle stelle con l'arte della cucina quantistica. Al centro del piatto brilla un arrosto di Carne di Balena Spaziale, sapientemente cotto
        utilizzando microonde entropiche sincronizzate per ottenere una superficie croccante e dorata che racchiude un cuore succulento. 
        Accanto, la carne di Mucca, teneramente preparata con la cottura a vapore termocinetica multipla, si fonde armoniosamente con funghi dell'Etere
        fluttuanti, esaltando un delicato profumo che solletica i sensi. Un contorno di Riso di Cassandra cuoce con vapore a flusso di particelle
        isoarmoniche, ogni singolo chicco brilla come una stella danzante, conservando i suoi nutrienti e colori translucidi. 
        Il sapore imprevedibile delle Shard di Materia Oscura è catturato in un'infusione sublime, avvolgendo tutto in un'atmosfera di meraviglia
        gravitazionale. Sorprendetevi con delle finissime teste di Idra, immerse in un consommé rigenerativo, la cui preparazione segue tecniche di taglio
        a risonanza sonica rigenerativa per assicurarne la sicurezza e il sapore impeccabile. 
        A completare questa opera d'arte, il nostro Pane di Luce irradia energia dorata, pronto a raccogliere ogni sapore con la sua morbidezza. E, come
        culmine di questo viaggio stellare, una degustazione di Biscotti della Galassia ruota allegramente intorno al piatto; scaglie di stelle e zucchero
        cosmico, accanto a un velato sentore di spezie Melange, che prolungano l'esperienza con un dolce indugio presciente che eleva l'anima oltre le
        stelle. 
        Ingredienti 
        Shard di Materia Oscura
        Carne di Balena spaziale
        Carne di Mucca
        Teste di Idra
        Riso di Cassandra
        Biscotti della Galassia
        Pane di LuceFunghi dell'Etere
        Spezie Melange 
        Tecniche 
        Cottura a Vapore con Flusso di Particelle Isoarmoniche
        Cottura a Vapore Termocinetica Multipla
        Taglio a Risonanza Sonica Rigenerativa
        Cottura con Microonde Entropiche Sincronizzate"""
    }
]



def extract_ingredients_techniques(data_partial_info:List[Dict]) -> List[Dict]:
    provider = Config.provider
    model = Config.model
    output_model_str = get_model_source("src.datamodels", "DishRecipe")

    output_full_info = []
    for data in tqdm(data_partial_info):
        recipe_text = data['recipe_raw_text']
        system_message = Config.system_message_template_recipes.format(
            output_model_str=output_model_str
        )
        message = Config.message_template_recipes.format(dish_recipe=recipe_text)

        try:
            response = call_llm(
                message=message,
                sys_message=system_message,
                model=f"{provider}:{model}",
                json_output=True,
            )
            response = json.loads(response)
            updated_data = {**data, **response}
            output_full_info.append(updated_data)
        except Exception as e:
            output_full_info.append({**data, "full_info": "Error"})
    return output_full_info


extract_ingredients_techniques(example_data)

ModuleNotFoundError: No module named 'src'